In [1]:
! pip install unsloth vllm
! pip install datasets evaluate rouge_score

# GRPO based Fine-Tuning Text2SQL

In [2]:
# Import necessary libraries
import torch  # PyTorch library for deep learning computations.
from unsloth import FastModel
from datasets import Dataset, DatasetDict, load_dataset, interleave_dataset # Used for loading and managing datasets.
from trl import GRPOConfig, GRPOTrainer
from transformers import TextStreamer
max_seq_length = 1024


import warnings # Used for managing warning messages.
warnings.filterwarnings("ignore") # Ignore warning messages during execution.

# Check if a CUDA-enabled GPU is available, otherwise use CPU for computations.
device: str = 'cuda' if torch.cuda.is_available() else 'cpu'

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-16 04:46:44 [__init__.py:239] Automatically detected platform cuda.


In [3]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

# Iterate through a list of English prompts
for prompt in ["Hello, How are you?", "My name is Midhun"]:
    # Print the current input prompt
    print("Input:", prompt)

    # Tokenize the input prompt and add translation instructions
    # Return PyTorch tensors and move them to the selected device
    inputTokens = tokenizer("translate English to French: {}".format(prompt), return_tensors="pt").to(device)

    # Generate the French translation using the model
    # Limit the output to a maximum of 50 tokens
    outputs = model.generate(inputTokens['input_ids'], attention_mask=inputTokens['attention_mask'], max_new_tokens=50)

    # Decode the generated output tokens into readable text and print the translation
    # Skip special tokens (like start and end tokens) in the output
    print("Output:", tokenizer.decode(outputs[0], skip_special_tokens=True))

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Input: Hello, How are you?
Output: translate English to French: Hello, How are you? I am a student, I need to ask you about my.

This is a bit complex, could you please rephrase that?

Here's a better approach:

"Bonjour, comment allez-vous? Je suis étudiant(e).
Input: My name is Midhun
Output: translate English to French: My name is Midhun and I want to share my experience with you.
I am a professional photographer with a passion for capturing images.
I have been working on a project that is very important to me.
My name is Midhun and I want to share my


### Dataset Prepration

In [ ]:
# Load dataset
dataset = load_dataset("b-mc2/sql-create-context", split='train')

In [ ]:
from pprint import pprint
print("-----Train Dataset----")
pprint(dataset[0])


-----Train Dataset----
{'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'context': 'CREATE TABLE head (age INTEGER)',
 'question': 'How many heads of the departments are older than 56 ?'}


In [ ]:
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = \
f"""You are SQL Engineer and for given context you will think and create SQL statments.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
system_prompt

'You are SQL Engineer and for given context you will think and create SQL statments.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

In [ ]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt + "Context:\n\n" + x["question"]},
        {"role": "user",   "content": x["question"]}
    ],
    "answer": x["answer"],
})
dataset[555]

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

{'answer': 'SELECT COUNT(*), affiliation FROM university WHERE enrollment > 20000 GROUP BY affiliation',
 'question': 'Find the number of universities that have over a 20000 enrollment size for each affiliation type.',
 'context': 'CREATE TABLE university (affiliation VARCHAR, enrollment INTEGER)',
 'prompt': [{'content': 'You are SQL Engineer and for given context you will think and create SQL statments.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>Context:\n\nFind the number of universities that have over a 20000 enrollment size for each affiliation type.',
   'role': 'system'},
  {'content': 'Find the number of universities that have over a 20000 enrollment size for each affiliation type.',
   'role': 'user'}]}

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

<re.Match object; span=(0, 71), match='<start_working_out>Let me think!<end_working_out>>

In [ ]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

In [ ]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

In [ ]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        # Correct answer gets 3 points!
        if guess == true_answer:
            score += 3.0
        # Match if spaces are seen
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # Penalize wrong answers
            except:
                score -= 0.5 # Penalize
        scores.append(score)
    return scores

In [ ]:
dataset[555]["prompt"]

[{'content': 'You are SQL Engineer and for given context you will think and create SQL statments.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>Context:\n\nFind the number of universities that have over a 20000 enrollment size for each affiliation type.',
  'role': 'system'},
 {'content': 'Find the number of universities that have over a 20000 enrollment size for each affiliation type.',
  'role': 'user'}]

In [ ]:
max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 2, # Set to 1 for a full training run
    max_steps = 50,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer

    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 78,577 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 0/999,885,952 (0.00% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately
1,0.000000,0.000000,0.000000,153.500000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,280.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,122.500000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,117.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,296.500000,0.000000,0.000000,0.000000
6,0.000000,1.000000,0.000000,152.000000,0.000000,0.000000,1.000000
7,-0.000000,0.500000,0.577350,65.750000,0.000000,0.000000,0.500000
8,0.000000,0.000000,0.000000,54.000000,0.000000,0.000000,0.000000
9,0.000000,-0.250000,0.500000,273.500000,0.000000,0.000000,-0.250000
10,0.000000,0.000000,0.000000,227.750000,0.000000,0.000000,0.000000


TrainOutput(global_step=50, training_loss=-1.7881393432617187e-09, metrics={'train_runtime': 673.3565, 'train_samples_per_second': 0.297, 'train_steps_per_second': 0.074, 'total_flos': 0.0, 'train_loss': -1.7881393432617187e-09})

In [ ]:
model.save_pretrained("gemma-3-text2SQL")  # Local saving
tokenizer.save_pretrained("gemma-3-text2SQL")

('gemma-3-text2SQL/tokenizer_config.json',
 'gemma-3-text2SQL/special_tokens_map.json',
 'gemma-3-text2SQL/tokenizer.model',
 'gemma-3-text2SQL/added_tokens.json',
 'gemma-3-text2SQL/tokenizer.json')

In [ ]:
messages = dataset[454]["prompt"]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)
# Create a dashed line for visual separation in the output
dash_line = '-'.join('' for x in range(100))
# Print the prompt, actual answer, and the model's generated answer
print(dash_line)
print(f'Message:\n{messages}')
print(dash_line)
print(f'Actual answer:\n{dataset[454]["answer"]}\n')
print(dash_line)

<start_working_out>
```sql
SELECT *
FROM Customers
WHERE State = 'NY';
```

<end_working_out><end_of_turn>
---------------------------------------------------------------------------------------------------
Message:
[{'content': 'You are SQL Engineer and for given context you will think and create SQL statments.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>Context:\n\nFind all the customer information in state NY.', 'role': 'system'}, {'content': 'Find all the customer information in state NY.', 'role': 'user'}]
---------------------------------------------------------------------------------------------------
Actual answer:
SELECT * FROM CUSTOMER WHERE State = "NY"

---------------------------------------------------------------------------------------------------


In [ ]:
! rm -r model*
! rm -r pef*

